In [ ]:
import numpy as np
import pickle
from pathlib import Path

from IPython.display import display
from lib.util_plot import *
from lib.constants import *
from lib.multilevel_analysis import *
from lib.pandas_impl import *
from lib.pandas_stats_VIS import *
from lib.matrix_analyzer import *
from lib.pandas_compute import *

import lib.FLY_preprocessing.fly_constants as FLY

### Data preparation

In [ ]:
data_folder = Path.cwd() / 'data' / 'FLY'
meta_folder = data_folder / 'meta'
eval_folder = Path.cwd() / 'data' / 'eval' / 'FLY_25-03-12'
eval_folder.mkdir(parents=True, exist_ok=True)
plot_folder = eval_folder / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
filename = eval_folder / "multilevel_analysis.pkl"
with open(filename, 'rb') as file:
    multilevel_analysis = pickle.load(file)

stats = multilevel_analysis.stats
df_summary = multilevel_analysis.df_summary

In [ ]:
pre_ids = set(df_summary.index.get_level_values("pre_id_mapped"))
post_ids = set(df_summary.index.get_level_values("post_id_mapped"))
all_ids = pre_ids.union(post_ids)
all_ids.remove(-1)   
no_presynaptic = all_ids - pre_ids

neuron_domain_pre = get_neuron_to_neuron_domain(df_summary, "pre_celltype", "post_celltype", \
                                                 celltype_order=None, ignored_neuron_ids=[-1])
neuron_domain_post = get_neuron_to_neuron_domain(df_summary, "pre_celltype", "post_celltype", \
                                                  celltype_order=None, ignored_neuron_ids=[-1])

In [ ]:
initPlotSettings(False)

In [ ]:
df_summary[(df_summary.index.get_level_values("pre_id_mapped") > -1) & (df_summary.index.get_level_values("post_id_mapped") > -1) & (df_summary[EMPIRICAL] > 0)]

In [ ]:
row_markers = {
    "pre_id_mapped" : [1]
}
col_markers = {
    "post_id_mapped" : [0]
}

### Overlapping/connected matrix

In [ ]:
matrix_analyzer = ConnectomeMatrixAnalyzer(df_summary, plot_folder)
matrix_analyzer.set_selection()
matrix_analyzer.set_data_columns(EMPIRICAL)
matrix_analyzer.build_matrix(["pre_celltype", "pre_id_mapped"], 
                             ["post_celltype", "post_id_mapped"],    
                             row_domains = neuron_domain_pre,  
                             col_domains = neuron_domain_post,
                             value_label_map = {},
                             aggregation_fn="sum",
                             default_value=-1)
matrix_analyzer.colormaps["binary"].set_under("white")
matrix_analyzer.render_matrix("FLY-overlapping-connected", 
                              colormap_name="binary", vmin=-1, vmax=1, normalization_function=None, 
                              row_markers=row_markers, col_markers=col_markers,
                              col_separator_lines=True, row_separator_lines=True, high_res=False)

### Specificity parameters matrix

In [ ]:
matrix_analyzer = ConnectomeMatrixAnalyzer(df_summary, plot_folder)
matrix_analyzer.set_selection(exclude_filter={})
matrix_analyzer.set_data_columns("model-C_preference")
matrix_analyzer.build_matrix(["pre_celltype", "pre_id_mapped"], 
                             ["post_celltype", "post_id_mapped"],    
                             row_domains = neuron_domain_pre,  
                             col_domains = neuron_domain_post,
                             value_label_map = {},
                             aggregation_fn="mean",
                             default_value=1)
matrix_analyzer.render_matrix("FLY-model-C-preference", 
                              colormap_name="coolwarm", vmin=-1, vmax=1, normalization_function=NormalizePreferenceValue(0, 8),
                              col_separator_lines=True, row_separator_lines=True, high_res=False)